In [ ]:
import datasets
train_txt_path = "../2_str_processor/nl_list_aozora.txt"
val_txt_path = train_txt_path
ds_train = datasets.load_dataset('text', data_files=[train_txt_path],split="train[:90%]")
ds_valid = datasets.load_dataset('text', data_files=[val_txt_path],split="train[90%:]")

text_datasets = datasets.DatasetDict({
    "train": ds_train,
    "valid": ds_valid,
})
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" #tokenizerの警告を消す

In [ ]:
print(text_datasets['train']['text'][0][:1000])

In [ ]:
from transformers import PreTrainedTokenizerFast
spm_folder = "../3_make_tokenizer/sentencepiece/"
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file = spm_folder + "spm_tokenizer.json",
)
print(tokenizer.encode("[NL]"),tokenizer.vocab_size)

In [ ]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        max_length=1024,
        truncation=True,
        return_overflowing_tokens=True,
        return_length=True,
    )
    #今回は1024トークンをフルで使うようにデータを詰める
    all_ids = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        all_ids+=input_ids

    buf_ids = []
    batch_list = []
    for tkn in all_ids:
        buf_ids.append(tkn)
        if len(buf_ids)==1024:
            batch_list.append(buf_ids)
            buf_ids = []
    return {"input_ids": batch_list}

tokenized_datasets = text_datasets.map(
    tokenize, batched=True, remove_columns=text_datasets["train"].column_names
)
tokenized_datasets.save_to_disk('./tokenized_datasets')